In [3]:
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
print(psycopg2.__version__) #Print the version to confirm


ImportError: DLL load failed while importing _psycopg: The specified module could not be found.

In [8]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))

In [10]:

from data_loading import load_data_from_postgres, load_data_using_sqlalchemy
from sql_queries import execute_telecom_queries
from data_preprocessing import handle_missing_values, treat_outliers, segment_users
from eda import perform_eda, correlation_analysis, perform_pca
from visualization import plot_histograms, plot_scatter, plot_correlation_matrix


ImportError: DLL load failed while importing _psycopg: The specified module could not be found.

In [2]:
# Preprocessing
xdr_df = handle_missing_values(xdr_df, strategy='mean') # Or other strategy
xdr_df = treat_outliers(xdr_df) # Implement your outlier handling method
xdr_df = segment_users(xdr_df, 'total_duration')


NameError: name 'handle_missing_values' is not defined

In [ ]:
# EDA
eda_results = perform_eda(xdr_df)
print(eda_results['descriptive_stats'])

In [ ]:
# Univariate Analysis (Graphical)
plot_histograms(xdr_df, ['session_duration', 'total_data'])
plt.show()


In [ ]:
# Bivariate Analysis
plot_scatter(xdr_df, 'Social Media data', 'total_data')
plt.show()


In [ ]:
# Correlation Analysis
correlation_matrix = correlation_analysis(xdr_df[['Social Media data', 'Google data', 'Email data', 'YouTube data', 'Netflix data', 'Gaming data', 'Other data']])
plot_correlation_matrix(correlation_matrix)
plt.show()


In [ ]:
# PCA
pca_results = perform_pca(xdr_df[['Social Media data', 'Google data', 'Email data', 'YouTube data', 'Netflix data', 'Gaming data', 'Other data']])
print(pca_results['explained_variance_ratio'])

In [ ]:
# Handset Analysis (Example using pandas; adapt for SQL if needed)
handset_counts = xdr_df['Handset Type'].value_counts().head(10)
print("\nTop 10 Handsets:\n", handset_counts)

In [ ]:
manufacturer_counts = xdr_df['Handset Manufacturer'].value_counts().head(3)
print("\nTop 3 Manufacturers:\n", manufacturer_counts)

In [ ]:
# Top 5 handsets per manufacturer (example)
for manufacturer in manufacturer_counts.index:
    print(f"\nTop 5 handsets for {manufacturer}:")
    print(xdr_df[xdr_df['Handset Manufacturer'] == manufacturer]['Handset Type'].value_counts().head(5))